In [1]:
# Cálcula el porcentaje de error entre dos series de máximos anuales.

import itertools
import os

import numpy as np
import pandas as pd
import xarray as xr

from matplotlib import pyplot as plt

import statsmodels.api        as sm
import statsmodels.stats.api  as sms

In [2]:
# Cantidad de celdas a eliminar del borde.
k = 1

# Archivos a evaluar.
eval = [ [0, 2], [0, 7], [4, 5], [5, 6], [8, 6], [9, 10], [11, 12], [13, 14],
    [15, 16], [13, 17], [14, 18], [17, 19], [14, 19], [13, 18], [13, 20],
    [20, 14] ]
n = 15
i = eval[n][0]
j = eval[n][1]
var = "wenzel"
#var = "medido"

name = [ "CHIRPS_interp_WRF", "CHIRPS_megalopolis", "prec_era5_hist_hist_days",
    "prec_hist_comp_estaciones", "prec_era5_hist_hist",
    "prec_mroc_hist_hist", "prec_mroc_2040_2040",
    "prec_mroc_hist_hist_days", "prec_mroc_h20a_hist",
    "prec_mroc_h20a_hist_coarsen_5", "prec_mroc_2040_2040_coarsen_5",
    "WRF_regrid_CHIRPS_days_qmap", "WRF_2040_regrid_CHIRPS_days_qmap",
    "WRF_regrid_ERA5_1985_2014_qmap", "WRF_regrid_ERA5_2040_2059_qmap",
    "WRF_regrid_ERA5_1985_2014", "WRF_regrid_ERA5_2040_2059",
    "WRF_regrid_ERA5_2080_2089_qmap",
    "WRF_regrid_ERA5_2040_2060_100PorcUrbano_qmap",
    "WRF_regrid_ERA5_2080_2089_100PorcUrbano_qmap",
    "WRF_regrid_ERA5_2040_2060_sinCUS_qmap"]

fname = f"{name[i]}_{name[j]}"
temp_d = "../../../temp/cc_idt/"
dir = f"{temp_d}comp/{fname}/"

# Si no existe la carpeta, la crea.
if not os.path.exists(f"{temp_d}comp/"): os.mkdir(f"{temp_d}comp/")
if not os.path.exists(dir): os.mkdir(dir)

if var == "wenzel":
    path_orig   = f"{temp_d}SAM/{name[i]}/{name[i]}_idf_valores.nc" 
    path_comp   = f"{temp_d}SAM/{name[j]}/{name[j]}_idf_valores.nc" 
if var == "medido":
    path_orig   = f"{temp_d}SAM/{name[i]}/{name[i]}_tretorno.nc" 
    path_comp   = f"{temp_d}SAM/{name[j]}/{name[j]}_tretorno.nc" 
path_orig_s = f"{temp_d}SAM/{name[i]}/{name[i]}_stats.nc" 
path_comp_s = f"{temp_d}SAM/{name[j]}/{name[j]}_stats.nc" 

ds_orig   = xr.open_dataset(path_orig  )
ds_comp   = xr.open_dataset(path_comp  )
orig_stat = xr.open_dataset(path_orig_s)
comp_stat = xr.open_dataset(path_comp_s)

if k != 0:
    ds_orig   =   ds_orig.isel(
        {"LONGITUD": slice(k, -k), "LATITUD": slice(k, -k)} )
    ds_comp   =   ds_comp.isel(
        {"LONGITUD": slice(k, -k), "LATITUD": slice(k, -k)} )    
    orig_stat = orig_stat.isel(
        {"LONGITUD": slice(k, -k), "LATITUD": slice(k, -k)} )
    comp_stat = comp_stat.isel(
        {"LONGITUD": slice(k, -k), "LATITUD": slice(k, -k)} ) 

vars = ["count", "mean", "std", "min", "25%", "50%", "75%", "max"]

#ds_comp["INTENSIDAD"].values = ds_comp["INTENSIDAD"].values[:, ::-1, :, :]

# Estadísticas de los valores originales.
ds_orig.to_dataframe()[["INTENSIDAD"]].describe()

,INTENSIDAD
count,78375.000000
mean,9.819153
std,8.781772
min,0.555836
25%,4.582871
50%,7.545643
75%,12.130344
max,238.204564


In [3]:
# Error total.
if var == "medido":
    ds_comp = ds_comp.drop_vars( ["AÑO", "PROBABILIDAD"] )
    ds_orig = ds_orig.drop_vars( ["AÑO", "PROBABILIDAD"] )
error = ( ( ds_comp - ds_orig )
    / ds_orig ).rename( {"INTENSIDAD": "P_ERROR"} )
#error = ( ( ds_comp.drop( ["AÑO", "PROBABILIDAD"] )
#    - ds_orig.drop( ["AÑO", "PROBABILIDAD"] ) )
#    / ds_orig ).rename( {"INTENSIDAD": "P_ERROR"} )
error.to_netcdf(dir + fname + ".nc")
df_e = error.to_dataframe()

tot = [ error["P_ERROR"].count(       ).values + 0,
        error["P_ERROR"].mean(        ).values + 0,
        error["P_ERROR"].std(         ).values + 0,
        error["P_ERROR"].min(         ).values + 0,
        error["P_ERROR"].quantile(0.25).values + 0,
        error["P_ERROR"].quantile(0.5 ).values + 0,
        error["P_ERROR"].quantile(0.75).values + 0,
        error["P_ERROR"].max(         ).values + 0 ]

print(f"Errores entre {name[i]} y {name[j]}")
for l in range( len(vars) ): print(f"{vars[l]}: {tot[l]:.2f}")

Errores entre WRF_regrid_ERA5_2040_2060_sinCUS_qmap y WRF_regrid_ERA5_2040_2059_qmap
count: 78375.00
mean: 0.03
std: 0.23
min: -0.68
25%: -0.11
50%: -0.00
75%: 0.12
max: 3.06
